In [3]:
import requests
import joblib
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from pandas import option_context
from tqdm import tqdm
from statsmodels.stats.inter_rater import aggregate_raters
from sklearn.metrics import cohen_kappa_score
from scipy.stats import mannwhitneyu,shapiro,ttest_ind

import cliffsDelta as cd
import GenerateActivities as gat
import important_features as imf

In [4]:
QUERY_ROOT = "https://api.github.com"
with open('token.txt', 'r') as file:
    TOKEN = file.read() # write your GitHub API key here
HEADERS={'Authorization':'token '+TOKEN}

## Section 3

### Question 1

#### Write an automated Python script to identify all GitHub bot actors in the list of contributors, by querying the GitHub REST API users endpoint and extracting the required information to make this decision. 

#### More information: You can search for the "type" key in the obtianed JSON result, If it is "Bot", then it is a bot actor, if it is "User", then it is an account.

In [5]:
# 1. Read the given <sample>.csv file into a pandas DataFrame (df).
# 2. Make sure there are 2 columns, one column name is "contributor" that has the name of the contributor, 
#    and another column name is "bothunter_type" that you will use later on in this assignment.
# 3. Display the DataFrame.

# YOUR CODE BELOW

# Read the csv file into a DataFrame
df = pd.read_csv('Contributors/sample18.csv')
df

,contributor,bothunter_type
0,Carifio24,Human
1,IAlibay,Human
2,JuliaTagBot,Bot
3,N-Dekker,Human
4,SoilRos,Human
5,Zethson,Human
6,andyfaff,Human
7,bebatut,Human
8,bioc-workshop-dev,Human
9,bsipocz,Human


In [6]:
# 1. Store the names of the contributors in a list
# 2. print the list

# YOUR CODE BELOW
contributors = df['contributor'].tolist()
print(contributors)

['Carifio24', 'IAlibay', 'JuliaTagBot', 'N-Dekker', 'SoilRos', 'Zethson', 'andyfaff', 'bebatut', 'bioc-workshop-dev', 'bsipocz', 'codecov-commenter', 'codspeed-hq[bot]', 'conda-bot', 'dalthviz', 'dask-bot', 'eendebakpt', 'galaxyproject-sentryintegration[bot]', 'garth-wells', 'github-actions[bot]', 'github-advanced-security[bot]', 'gxydevbot', 'hackmd-hub[bot]', 'hoxbro', 'jd-foster', 'jpdean', 'kenodegard', 'kwrobot', 'lrnv', 'matthewturk', 'mirror-dump', 'mnwhite', 'nickodell', 'pat-s', 'rafaelorozco', 'renovate[bot]', 'review-notebook-app[bot]', 'sathvikbhagavan', 'steppi', 'tupui', 'yarikoptic']


In [7]:
# 1. Using a for loop, iterate over each contributor in the list and query the GitHub Users API.
#    You can use "query = f'{QUERY_ROOT}/users/{contributor}'", where QUERY_ROOT is defined at the beginning of this notebook 
#    and 'contributor' is each individual contributor from the list
# 2. Get the response using 'response = requests.get(query, headers=HEADERS)'.
# 3. convert the response to JSON using 'json_response = response.json()'.
# 4. Iterate over each JSON response and get the value of the 'type' key. If it is "Bot" then the contributor is a bot actor, 
#    if "User" then the contributor is an account. You should boolean values to indicate if the contributor is a bot actor (True) or User/Organisation (False)
# 5. Save these results in list of dictionary of the form [{'contributor': <contributor name>, 'app': <boolean value>}, {...}, {...}, {...}].
#    Lets call this list as "app_dict"
# 6. Finally convert this list of dictionary to DataFrame by writing 'pd.DataFrame.from_dict(app_dict)'
# 7. Display the DataFrame. This should have two columns - contributor and app

# YOUR CODE BELOW
app_dict = list()
for contributor in contributors:
    query = f'{QUERY_ROOT}/users/{contributor}'
    response = requests.get(query, headers=HEADERS)
    json_response = response.json()
    if json_response.get('type') == 'Bot':
        app = True
    else:
        app = False
    app_dict.append({'contributor': contributor, 'app': app})

app_df = pd.DataFrame.from_dict(app_dict)
app_df

,contributor,app
0,Carifio24,False
1,IAlibay,False
2,JuliaTagBot,False
3,N-Dekker,False
4,SoilRos,False
5,Zethson,False
6,andyfaff,False
7,bebatut,False
8,bioc-workshop-dev,False
9,bsipocz,False


### Question 2

#### Add a new 'actor' column in the CSV file to store this information. You can write True if it is a bot actor and False if it is not.

#### Report on the total number of GitHub bot actors, and User accounts present in the list of accounts of your dataset.

In [8]:
# Merge the app DataFrame to df by writing 'pd.merge(df, <app df>, on='contributor'). This is similar to SQL join on primary key 'contributor'.
# The resultant df should have 3 columns - contributor, bothunter_type and user_type.

# YOUR CODE BELOW

merge_df = pd.merge(df, app_df, on='contributor')
# rename app column to actor
merge_df.rename(columns={'app': 'user_type'}, inplace=True)
merge_df

,contributor,bothunter_type,user_type
0,Carifio24,Human,False
1,IAlibay,Human,False
2,JuliaTagBot,Bot,False
3,N-Dekker,Human,False
4,SoilRos,Human,False
5,Zethson,Human,False
6,andyfaff,Human,False
7,bebatut,Human,False
8,bioc-workshop-dev,Human,False
9,bsipocz,Human,False


### Question 3

#### For each bot actor, write their purpose, the task that they automate by looking at their GitHub profile, homepage, information on the GitHub Marketplace, or other documentation that you can find online.


In [9]:
bots = merge_df.query('user_type == True')
print("CodSpeed HQ is a continuous benchmarking platform designed to track and compare the performance of codebases during development. It uses a smart runtime engine to measure performance accurately and reproducibly, without creating a significant runtime overhead, unlike traditional benchmarks.")
print("galaxyproject-sentryintegration[bot]	does not appear to have documentation on the internet.")
print("The GitHub Actions bot serves various purposes in GitHub Actions workflows. It can be used to perform automated tasks such as assigning users to issues, creating pull requests, and managing repository operations. For example, you can use the bot to assign users to pull requests by calling the GitHub API with the token generated by GitHub Actions in the GITHUB_TOKEN secret.")
print("GitHub Advanced Security (GHAS) is a suite of security features designed to enhance the security of code in GitHub repositories. It is particularly aimed at supporting the DevSecOps approach by integrating security practices early in the Software Development Life Cycle (SDLC).")
print("""
 HackMD Hub is a GitHub app that allows users to collaborate on Markdown files and sync them with GitHub repositories.
 It provides a way for teams to work together on documentation in real-time, making discussions and processes more visible and accessible.
 This tool is particularly useful for open-source projects and teams looking to enhance their documentation workflow.
""")
print("""Renovate is an automated dependency update tool for GitHub that helps to update dependencies in your code without needing to do it manually.
 It scans repositories to find references to dependencies and creates pull requests to update versions automatically if newer versions are available.
 Renovate is free to install for both public and private repositories and is provided complimentary of Mend (formerly known as WhiteSource).""")

print("""The Review Notebook App on GitHub is designed to provide a complete code review workflow for Jupyter Notebooks, making it easier to manage version control, code reviews, and collaboration within teams. It offers visual diffs for every GitHub commit or Pull Request containing Notebooks and allows commenting on the Notebook diff to review Jupyter Notebook PRs.""")
bots

CodSpeed HQ is a continuous benchmarking platform designed to track and compare the performance of codebases during development. It uses a smart runtime engine to measure performance accurately and reproducibly, without creating a significant runtime overhead, unlike traditional benchmarks.
galaxyproject-sentryintegration[bot]	does not appear to have documentation on the internet.
The GitHub Actions bot serves various purposes in GitHub Actions workflows. It can be used to perform automated tasks such as assigning users to issues, creating pull requests, and managing repository operations. For example, you can use the bot to assign users to pull requests by calling the GitHub API with the token generated by GitHub Actions in the GITHUB_TOKEN secret.
GitHub Advanced Security (GHAS) is a suite of security features designed to enhance the security of code in GitHub repositories. It is particularly aimed at supporting the DevSecOps approach by integrating security practices early in the So

,contributor,bothunter_type,user_type
11,codspeed-hq[bot],Bot,True
16,galaxyproject-sentryintegration[bot],Bot,True
18,github-actions[bot],Bot,True
19,github-advanced-security[bot],Bot,True
21,hackmd-hub[bot],Bot,True
34,renovate[bot],Bot,True
35,review-notebook-app[bot],Bot,True


## Section 4

### Question 1

#### Write an automated script to use BIMBAS (Bot Identification Model Based on Activity Sequences) to obtain a prediction of the GitHub contributor type (Bot or Human) for the contributors present in your dataset.

#### Folow the steps given in each cell to use BIMBAS and obtain predictions.

### Read all the events and select the events performed by the contributors present in the given set

In [10]:
# 1. Read the csv file of events (all_events.csv) provided along with this notebook
# 2. Convert created_at column to datetime format 
#    One possible way is to use lambda function: "events.assign(created_at=lambda d: pd.to_datetime(d.created_at, unit='ms'))"
# 3. Get the contributors provided to you in a list or use the contributors list that you created in Section 3 Question 1. 
#    e.g., ['contributor1', 'contributor2', 'contributor3',...]
# 4. Select the events performed by these contributors alone. Do not consider all the events. You should consider only the events
#    performed by the contributors provided to you. Hint: you can use the df.query function
# 5. Display the considered events.

# YOUR CODE BELOW
events = pd.read_csv('all_events.csv')
events['created_at'] = pd.to_datetime(events['created_at'], unit='ms')
contributors_list = contributors
selected_events = events.query('login in @contributors_list')
print(selected_events.shape)
selected_events

(99977, 25)


,Unnamed: 0,event_id,event_type,login,repository,created_at,action,PR_number,state,PR_node_id,...,comment_node_id,merged,ref,ref_type,issue_number,issue_node_id,issue_closed_at,tag_name,release_node_id,org
6,6,39747396929,CommitCommentEvent,github-actions[bot],QuantEcon/lecture-python-intro,2024-07-01 00:01:47,NaN,NaN,NaN,NaN,...,CC_kwDOIy5DFs4IkKwX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QuantEcon
15,15,39747413489,CommitCommentEvent,github-actions[bot],QuantEcon/lecture-python-intro,2024-07-01 00:03:24,NaN,NaN,NaN,NaN,...,CC_kwDOIy5DFs4IkKw0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QuantEcon
16,16,39747416135,PushEvent,github-actions[bot],napari/npe2api,2024-07-01 00:03:40,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,napari
21,21,39747427332,CreateEvent,github-actions[bot],SciML/PubChem.jl,2024-07-01 00:04:44,NaN,NaN,NaN,NaN,...,NaN,NaN,compathelper/new_version/2024-07-01-00-04-42-9...,branch,NaN,NaN,NaN,NaN,NaN,SciML
29,29,39747452010,IssueCommentEvent,JuliaTagBot,SciML/CommonWorldInvalidations.jl,2024-07-01 00:07:04,created,NaN,closed,NaN,...,IC_kwDOMOEgaM6DD0Tk,NaN,NaN,NaN,5.0,I_kwDOMOEgaM6N6wg8,2024-06-28T18:51:28Z,NaN,NaN,SciML
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358391,358391,42415303969,PushEvent,github-actions[bot],SciML/LineSearch.jl,2024-09-30 23:24:21,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SciML
358397,358397,42415358855,IssueCommentEvent,codspeed-hq[bot],conda/conda,2024-09-30 23:27:57,created,14285.0,open,PR_kwDOAF8kJs59LhsL,...,IC_kwDOAF8kJs6OHzZU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conda
358401,358401,42415474205,PushEvent,github-actions[bot],Bioconductor/gha-build-jupyter-release-amd64,2024-09-30 23:34:53,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bioconductor
358403,358403,42415512378,PushEvent,github-actions[bot],napari/npe2api,2024-09-30 23:37:12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,napari


### Execute BIMBAS to obtain the predictions

In [11]:
# Do not modify this cell
# Use the following function the code below to obtain your predictions

def execute_bimbas(contributors_list, selected_events):
    '''
    args: contributors_list (list) - list of contributors in the dataset provided to you
          selected_events (DataFrame) - DataFrame of events performed by the considered contributors
    return: bimbas_prediction (DataFrame) - DataFrame of preditions along with the confidence in prediction for each contributor
    '''
    result=pd.DataFrame()
    temp_list = []
    bimbas = joblib.load('bimbas.joblib')
    date_limit = pd.to_datetime(selected_events.created_at.max()) + pd.DateOffset(-90)
    
    for contributor in tqdm(contributors_list):
        # c = contributors[contributor]
        activities = gat.activity_identification(selected_events.query('login==@contributor and created_at>=@date_limit'))
        activity_features = (
                            imf.extract_features(activities)
                            .set_index([[contributor]])
                            )
        # features = pd.concat([features,activity_features])
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            probability = bimbas.predict_proba(activity_features)[0][1]
        if(probability <= 0.5):
            contributor_type = 'Human'
        else:
            contributor_type = 'Bot'
        confidence = (abs(probability - 0.5)*2).round(3)
    
        pred = activity_features.set_index([[contributor]]).assign(
            prediction=contributor_type,
            confidence = confidence,
            )
        temp_list.extend([{'contributor':contributor,'bimbas_type':contributor_type,'confidence':confidence}])
    
    bimbas_prediction = pd.DataFrame.from_dict(temp_list)

    return(bimbas_prediction)

In [12]:
# Write your code here to call execute_bimbas function. 
# Pass the contributors list and the filtered events DataFrame as argument to "execute_bimbas" function
# Prediction returned by BIMBAS will have "contributor", "bimbas_type" and "confidence"
# Note: Print the prediction provided by BIMBAS - else this cell will not be graded.

# YOUR CODE BELOW

predictions = execute_bimbas(contributors_list, selected_events)
predictions

100%|██████████| 40/40 [00:05<00:00,  6.82it/s]


,contributor,bimbas_type,confidence
0,Carifio24,Human,0.825
1,IAlibay,Human,0.802
2,JuliaTagBot,Bot,0.345
3,N-Dekker,Human,0.754
4,SoilRos,Bot,0.867
5,Zethson,Human,0.819
6,andyfaff,Human,0.766
7,bebatut,Human,0.890
8,bioc-workshop-dev,Bot,0.887
9,bsipocz,Human,0.663


### Create a column in your predictions DataFrame to have predictions provided by BIMBAS. 
### Now your DataFrame should have the following columns - contributor, bothunter_type, app, bimbas_type, confidence 

### Merging bothunter and bimbas predictions

In [13]:
# Merge the predictions provided by BIMBAS to the resultant dataset of Section 3. This final DataFrame should have the following columns - 
# contributor, bothunter_type, user_type, bimbas_type, and confidence.

merged_prediction_df = pd.merge(merge_df, predictions, on='contributor')
merged_prediction_df

,contributor,bothunter_type,user_type,bimbas_type,confidence
0,Carifio24,Human,False,Human,0.825
1,IAlibay,Human,False,Human,0.802
2,JuliaTagBot,Bot,False,Bot,0.345
3,N-Dekker,Human,False,Human,0.754
4,SoilRos,Human,False,Bot,0.867
5,Zethson,Human,False,Human,0.819
6,andyfaff,Human,False,Human,0.766
7,bebatut,Human,False,Human,0.890
8,bioc-workshop-dev,Human,False,Bot,0.887
9,bsipocz,Human,False,Human,0.663


### Question 2
#### Using Cohen's Kappa compute and report the interrater agreement score between the labels computed by both the bot identification approaches. Mention your interpretation of Cohen's Kappa

In [14]:
# Hint: use the cohen_kappa_score library
# YOUR CODE BELOW

from sklearn.metrics import cohen_kappa_score

kappa_score = cohen_kappa_score(merged_prediction_df['bothunter_type'], merged_prediction_df['bimbas_type'])
kappa_score

# The kappa score is 0.5789473684210527, which means that there is a moderate level of agreement between the two methods.
# the perfect agreement would be 1, and no agreement would be 0.

np.float64(0.5789473684210527)

### Question 3
#### Determine the final type of each contributor. Whenever user_type column has the value "User" check if both bimbas_type and bothunter_type give the same prediction, then consider it as your final prediction in "acc_type" column.
#### For the contributors that have different predictions, i.e. bimbas_type is not same as bothunter_type, then make a manual verification and consider that as the type for that contributor. Add an extra column to the DataFrame ﬁle called 'manual' that has the label determined by you. 
#### For manual verification you can make use the GitHub UI and the GitHub API https://api.github.com/users/username

#### Whenever user_type column has the value "Bot", directly write your final acc_type as "Bot Actor"

#### More information:
For each contributor, you can look at their activities in GitHub UI, look at their latest events using GitHub Events API - https://api.github.com/users/<contributor>/events, to make a decision on their type.

In addition, you can also write a very small reason why do you think they are bot or human (e.g., same activity at regular interval so Bot, files committed and code modified looks like Human, comments look like Human, comment look like Bot, test report looks automated so Bot, and so on....). This will be useful to answer your next question.

In [15]:
# Write the code for your manual decision here
# example : 
# manual_list_dict = [{'contributor': '<contributor name>', 'manual': '<your prediction>'}, {....}, {....}, {....}]
# pd.DataFrame.from_dict(manual_list_dict)

# YOUR CODE BELOW

# if they are both 'Human' then set 'acc_type' as 'Human' and if both are 'Bot' then set 'acc_type' as 'Bot Actor'.
# if they are different we temporary set 'acc_type' as 'Manual'.
bot_deter_df = merged_prediction_df.copy()
bot_deter_df['acc_type'] = bot_deter_df.apply(lambda row: "Manual" if row['bimbas_type']!= row['bothunter_type'] else row['bimbas_type'], axis=1)
bot_deter_df['acc_type'] = bot_deter_df.apply(lambda row: "Bot" if 'bot' in row['contributor'].lower() else row["acc_type"], axis=1)
bot_deter_df

,contributor,bothunter_type,user_type,bimbas_type,confidence,acc_type
0,Carifio24,Human,False,Human,0.825,Human
1,IAlibay,Human,False,Human,0.802,Human
2,JuliaTagBot,Bot,False,Bot,0.345,Bot
3,N-Dekker,Human,False,Human,0.754,Human
4,SoilRos,Human,False,Bot,0.867,Manual
5,Zethson,Human,False,Human,0.819,Human
6,andyfaff,Human,False,Human,0.766,Human
7,bebatut,Human,False,Human,0.890,Human
8,bioc-workshop-dev,Human,False,Bot,0.887,Manual
9,bsipocz,Human,False,Human,0.663,Human


In [16]:
# take the rows that have a different bothunter_type and bimbas_type.
diff_df = bot_deter_df[bot_deter_df['acc_type'] == "Manual"]
diff_df

,contributor,bothunter_type,user_type,bimbas_type,confidence,acc_type
4,SoilRos,Human,False,Bot,0.867,Manual
8,bioc-workshop-dev,Human,False,Bot,0.887,Manual
27,lrnv,Human,False,Bot,0.910,Manual
29,mirror-dump,Human,False,Bot,0.786,Manual
33,rafaelorozco,Human,False,Bot,0.864,Manual


In [17]:
# Write your code below here to determine the final type of contributor.
# add column named 'acc_type' and write your final decision in it. Final decision is majority of three types - bimbas_type, bothunter_type, manual.

print("The user SoilRos has a blog linked to his github account so it's surely a real person")
print("The user bioc-workshop-dev made commits on the same period so it's likely a real person.")
print("The user lrnv is most probably a real person as he did commit linearly, has a photo and several public repositories.")
print("The user mirror-dump	made forks, has no photo and no activities in commit. It looks like a bot.")
print("The user rafaelorozco is in reality Rafael Orozco, Computational Sciences and Engineering Phd Candidate")
to_change = ['SoilRos', 'bioc-workshop-dev', 'lrnv', 'rafaelorozco']

# Replace acc_type of theseusers according to the manual researches.
bot_deter_df['Manual'] = bot_deter_df.apply(lambda row: "Bot" if 'mirror-dump' == row['contributor'] else row["acc_type"], axis=1)
bot_deter_df['Manual'] = bot_deter_df.apply(lambda row: "Human" if row['contributor'] in to_change else row["acc_type"], axis=1)
bot_deter_df

The user SoilRos has a blog linked to his github account so it's surely a real person
The user bioc-workshop-dev made commits on the same period so it's likely a real person.
The user lrnv is most probably a real person as he did commit linearly, has a photo and several public repositories.
The user mirror-dump	made forks, has no photo and no activities in commit. It looks like a bot.
The user rafaelorozco is in reality Rafael Orozco, Computational Sciences and Engineering Phd Candidate


,contributor,bothunter_type,user_type,bimbas_type,confidence,acc_type,Manual
0,Carifio24,Human,False,Human,0.825,Human,Human
1,IAlibay,Human,False,Human,0.802,Human,Human
2,JuliaTagBot,Bot,False,Bot,0.345,Bot,Bot
3,N-Dekker,Human,False,Human,0.754,Human,Human
4,SoilRos,Human,False,Bot,0.867,Manual,Human
5,Zethson,Human,False,Human,0.819,Human,Human
6,andyfaff,Human,False,Human,0.766,Human,Human
7,bebatut,Human,False,Human,0.890,Human,Human
8,bioc-workshop-dev,Human,False,Bot,0.887,Manual,Human
9,bsipocz,Human,False,Human,0.663,Human,Human


### Question 4
#### Study and report the purpose of these identified bots and humans in the repository assigned to you.
#### For example, the purpose can be based on but not limited to - 
##### 1) the type of activities that they are performing (releasing a version on every Sunday, updating the documentation), 
##### 2) type of comments they are posting (reviewing code, test summary report), 
##### 3) when are they triggered (when a new PR is created, when someone in the project ask the bot to merge the code) and so on.  
#### Note: 
##### The purpose can be other than what is provided in examples above.

## Section 5

### Question 1
#### Use the filtered events file that has the events performed by the contributors provided to you
#### Group each event into the following four categories

Issues: IssueCommentEvent, IssuesEvent  
Pull Requests: PullRequestEvent, PullRequestReviewCommentEvent  
Commits: CommitCommentEvent, PushEvent  
Repository: CreateEvent, DeleteEvent, ForkEvent, GollumEvent, MemberEvent, PublicEvent, ReleaseEvent, SponsorshipEvent, WatchEvent  

#### Hint:
1. Add a column called event_group that mentions which event group does that event belong to. Each event (row) should correspond to an event group.  
2. Then perform groupby on ['login','event_group'],  
3. use .agg (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html) to count the number of events performed by each contributor in each group,  
4. use pivot with the required arguments (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html). An example is given [here](#pivot_example) in question 2(b), and
5. Reset index and rename axis with None
7. Finally merge it with your 'acc_type' field from the DataFrame you created in Section4, drop contributor, and fillna with 0
8. The final DataFrame should have the following columns - login, PR, commit, issue, repo, acc_type

The final DataFrame should be looking like the following  
![](event_group.png)

In [18]:
# In the resultant DataFrame - each row should corespond to a contributor, and the columns should have all the event groups  
# and the type of the contributor (that you decided in the previous DataFrame) and the values should be the number 
# of events of that event event group the contriubutor has performed.

issue_group = ['IssueCommentEvent', 'IssuesEvent']
pr_group = ['PullRequestEvent', 'PullRequestReviewCommentEvent']
commit_group = ['CommitCommentEvent', 'PushEvent']
repo_group = ['CreateEvent', 'DeleteEvent', 'ForkEvent', 'GollumEvent', 'MemberEvent', 'PublicEvent', 'ReleaseEvent', 'SponsorshipEvent', 'WatchEvent']

In [46]:
# YOUR CODE BELOW
# For each contributor, count the number of events in each event group and create a dataframe that contains the contributor name and the count of events in each group.
contributor_events = []
for contributor in contributors:
    # Filter events for the current contributor
    current_contributor_events = events.where(lambda row: row['login'] == contributor)

    # Group by event group and count the number of events
    count = current_contributor_events["event_type"].value_counts()

    count_dict = count.to_dict()
    count_dict["contributor"] = contributor

    # Add the contributor name and the count of events in the DataFrame
    contributor_events.append(count_dict)

count_df = pd.DataFrame.from_dict(contributor_events)
count_df

,PullRequestEvent,PushEvent,CreateEvent,ForkEvent,IssueCommentEvent,PullRequestReviewCommentEvent,PullRequestReviewEvent,DeleteEvent,contributor,IssuesEvent,WatchEvent,ReleaseEvent,GollumEvent,CommitCommentEvent
0,14.0,12.0,3.0,2.0,2.0,1.0,1.0,1.0,Carifio24,NaN,NaN,NaN,NaN,NaN
1,29.0,42.0,16.0,NaN,58.0,28.0,38.0,11.0,IAlibay,12.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,599.0,NaN,NaN,NaN,JuliaTagBot,8.0,NaN,NaN,NaN,NaN
3,29.0,2.0,NaN,1.0,50.0,80.0,86.0,NaN,N-Dekker,2.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,SoilRos,NaN,NaN,NaN,NaN,NaN
5,26.0,40.0,10.0,NaN,54.0,21.0,29.0,11.0,Zethson,27.0,1.0,NaN,NaN,NaN
6,24.0,9.0,NaN,NaN,114.0,12.0,9.0,NaN,andyfaff,5.0,NaN,NaN,NaN,NaN
7,50.0,44.0,14.0,1.0,20.0,53.0,74.0,12.0,bebatut,12.0,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,bioc-workshop-dev,13.0,NaN,NaN,NaN,NaN
9,143.0,90.0,1.0,1.0,195.0,74.0,171.0,20.0,bsipocz,64.0,NaN,NaN,NaN,NaN


In [20]:
# YOUR CODE BELOW


### Question 2 (a)

#### Compute the median number of events per event group for Bot+Bot actors and Humans and write in DataFrame.

Row should correspond to type (Bot_BotActor and Human), Column should have Event group name and the values should be the median value of Bot_BotActor or Human for that particular event group. An example is given below

In [21]:
# For example:
medians = [{'event_group': 'event_group1', 'median': 'val1', 'acc_type': 'Bot_app'}, 
           {'event_group': 'event_group1', 'median': 'val2', 'acc_type': 'Human'},
           {'event_group': 'event_group2', 'median': 'val3', 'acc_type': 'Bot_app'},
           {'event_group': 'event_group2', 'median': 'val4', 'acc_type': 'Human'},
           {'event_group': 'event_group3', 'median': 'val5', 'acc_type': 'Bot_app'},
           {'event_group': 'event_group3', 'median': 'val6', 'acc_type': 'Human'},
           {'event_group': 'event_group4', 'median': 'val7', 'acc_type': 'Bot_app'},
           {'event_group': 'event_group5', 'median': 'val8', 'acc_type': 'Human'}]
df_medians = pd.DataFrame.from_dict(medians)
df_medians

,event_group,median,acc_type
0,event_group1,val1,Bot_app
1,event_group1,val2,Human
2,event_group2,val3,Bot_app
3,event_group2,val4,Human
4,event_group3,val5,Bot_app
5,event_group3,val6,Human
6,event_group4,val7,Bot_app
7,event_group5,val8,Human


In [71]:
# YOUR CODE BELOW
# for each event group, calculate the median number of events for Bot and Human actors.
count_df['acc_type'] = bot_deter_df['acc_type']
human_df = count_df.where(lambda row: row['acc_type'] == 'Human')
bot_df = count_df.where(lambda row: row['acc_type'] == 'Bot')
bot_event_dict = bot_df.drop(['contributor', 'acc_type'], axis=1, inplace=False).sum(skipna=True).to_dict()
human_event_dict = human_df.drop(['contributor', 'acc_type'], axis=1, inplace=False).sum(skipna=True).to_dict()
median_df = pd.DataFrame.from_dict([bot_event_dict, human_event_dict])
median_df.describe()

,PullRequestEvent,PushEvent,CreateEvent,ForkEvent,IssueCommentEvent,PullRequestReviewCommentEvent,PullRequestReviewEvent,DeleteEvent,IssuesEvent,WatchEvent,ReleaseEvent,GollumEvent,CommitCommentEvent
count,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000,2.00000,2.000000,2.000000,2.00000,2.000000,2.00000
mean,1131.500000,43121.000000,626.000000,14.000000,2446.500000,704.00000,827.000000,210.50000,289.500000,2.500000,211.00000,3.500000,308.00000
std,12.020815,58914.722795,560.028571,7.071068,1448.861795,412.95036,753.775829,144.95689,118.086832,3.535534,265.87215,4.949747,432.74935
min,1123.000000,1462.000000,230.000000,9.000000,1422.000000,412.00000,294.000000,108.00000,206.000000,0.000000,23.00000,0.000000,2.00000
25%,1127.250000,22291.500000,428.000000,11.500000,1934.250000,558.00000,560.500000,159.25000,247.750000,1.250000,117.00000,1.750000,155.00000
50%,1131.500000,43121.000000,626.000000,14.000000,2446.500000,704.00000,827.000000,210.50000,289.500000,2.500000,211.00000,3.500000,308.00000
75%,1135.750000,63950.500000,824.000000,16.500000,2958.750000,850.00000,1093.500000,261.75000,331.250000,3.750000,305.00000,5.250000,461.00000
max,1140.000000,84780.000000,1022.000000,19.000000,3471.000000,996.00000,1360.000000,313.00000,373.000000,5.000000,399.00000,7.000000,614.00000


### Question 2 (b)

Plot a heatmap of the DataFrame using seaborn - 
1. First convert the dataframe to the required format using pivot, example is given below
2. plot using seaborn - sns.heatmap(df_medians, annot=True, vmin=0, vmax=300, cmap="crest"). More details: https://seaborn.pydata.org/generated/seaborn.heatmap.html)

#### pd.pivot example:
<a id='pivot_example'></a>

In [23]:
# Main DataFrame
medians = [{'event_group': 'event_group1', 'median': 'val1', 'acc_type': 'Bot'}, 
           {'event_group': 'event_group1', 'median': 'val2', 'acc_type': 'Human'},
           {'event_group': 'event_group2', 'median': 'val3', 'acc_type': 'Bot'},
           {'event_group': 'event_group2', 'median': 'val4', 'acc_type': 'Human'},
           {'event_group': 'event_group3', 'median': 'val5', 'acc_type': 'Bot'},
           {'event_group': 'event_group3', 'median': 'val6', 'acc_type': 'Human'},
           {'event_group': 'event_group4', 'median': 'val7', 'acc_type': 'Bot'},
           {'event_group': 'event_group5', 'median': 'val8', 'acc_type': 'Human'}]
df_medians = pd.DataFrame.from_dict(medians)
df_medians

,event_group,median,acc_type
0,event_group1,val1,Bot
1,event_group1,val2,Human
2,event_group2,val3,Bot
3,event_group2,val4,Human
4,event_group3,val5,Bot
5,event_group3,val6,Human
6,event_group4,val7,Bot
7,event_group5,val8,Human


In [24]:
# pivot the main DataFrame
df_medians.pivot(index='acc_type', columns='event_group', values='median')#.reset_index().rename_axis(None,axis=1)

event_group,event_group1,event_group2,event_group3,event_group4,event_group5
acc_type,,,,,
Bot,val1,val3,val5,val7,NaN
Human,val2,val4,val6,NaN,val8


In [25]:
# YOUR CODE BELOW

In [26]:
# YOUR CODE BELOW


#### What is the difference that you observe between Bots+Bot actors and Humans?

#### What is the differnce that you observe between Event groups?

#### What is the difference that you observe between Bots+Bot actors and Humans and Event groups all considering at the same time?

### Question 3

#### Create boxen plots to visualise the distribution of number of events in each event group. 
#### For more information you can visit - https://seaborn.pydata.org/generated/seaborn.boxenplot.html#seaborn.boxenplot 
#### You should highlight the data points that correspond to bots using a stripplot in seaborn. https://seaborn.pydata.org/tutorial/categorical.html#categorical-tutorial  
#### Interpret the results of the visualisation.

In [27]:
# YOUR CODE BELOW - Visualize number of events from pull request event group


In [28]:
# YOUR CODE BELOW - Visualise number of events from issue event group


In [29]:
# YOUR CODE BELOW - Visualise number of events from commit event group


In [30]:
# YOUR CODE BELOW - Visualise number of events from repo event group


### Question 4.1

#### Statistical identify whether number of events in each event group is normally distributed or not.
#### Null hypothesis - $H_0$: Sample comes from the data that has normal distribution.
#### Use Shapiro-Wilk test for this purpose. Use the p-value with a threshold of 0.05 to determine whether $H_0$ can be rejected with statistical significance or not. 

#### Use shapiro from scipy.stats to perform this test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html)

In [31]:
# YOUR CODE BELOW

### Question 4.2
#### Determine if there is any statistical diﬀerence in number of events between the identified event groups. Perform this test on all pairs of event groups. 
#### Null hypothesis - $H_0$: Any two event group come from the same population.
#### If $H_0$ is rejected in Shapiro-Wilk test (at least for one of the two event groups considered for test), use the Mann-Whitney U statistical test for this purpose. If $H_0$ is not rejected in Shapiro-Wilk test, use the independent t-test for this purpose.  
#### In any case, use the p-value with a threshold of 0.01 to determine whether $H_0$ can be rejected with statistical significance. 

#### Use mannwhitneyu from scipy.stats to perform Mann-Whitney U test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html) 
#### or 
#### ttest_ind from scipy.stats to perform independent t-test https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html.

#### More information:

You can pass the following arguments to mannwhitneyu from scipy.stats - (method='exact', nan_policy='omit'). For ttest_ind you can use - (nan_policy='omit')

In [32]:
# YOUR CODE BELOW

### Question 4.3
#### Each time you reject the null hypothesis $H_0$, quantify the effect size of the diﬀerence between the groups using cliﬀ’s delta ($\delta$). 
#### To calculate cliﬀ’s delta, you can pass the list of values to cliﬀ delta.py file given in the repository. E.g., cliffsDelta.cliffsDelta(list of values, list of values). This will return the effect size.
#### Refer to the table given in the TP document and mention your interpretation (negligible, small, medium, large).

In [33]:
# YOU CODE BELOW